반드시 들어가야 하는 내용 (Mission)
앞서서 여러분에게 최대한의 자유를 드린다고 했지만, 그래도 발표에 반드시 포함되어야 하는 내용은 다음과 같습니다.

1-1지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.
1. 지역은 어떻게 구분했나요?
2. 지역별로 선호하는 게임의 장르를 파악했나요?
3. 지역별로 선호하는 게임의 장르가 왜 같다고/다르다고 생각했나요?

1-2 연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.
1. 트렌드는 어떤 것을 기준으로 파악했나요?
2. 연도별로 트렌드가 존재하나요?
3. 왜 존재한다/존재하지 않는다고 생각했나요?

1-3.인기가 많은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다.
1. 인기가 많다는 것을 어떻게 정의했나요?
2. 해당 게임이 왜 인기가 많다고 생각했나요?
3. 시각화를 통해 무엇을 나타내고자 하는지를 고려했나요?
4. 해당 분석을 통해 어떤 인사이트를 도출할 수 있나요?

1-4. 다음 분기에 어떤 게임을 설계해야 하는지에 대한 결론이 제시되어야 합니다.

더불어 유의미한 인사이트를 도출하기 위해서는

- 다음 분기가 언제인지
- 회사의 규모는 어느정도인지 와 같은 디테일한 상황 설정과 더불어 '왜' 이러한 결론을 도출하였는지에 대한 고민이 필요합니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df = pd.read_csv("vgames2.csv")

Name : 게임의 이름입니다.

Platform : 게임이 지원되는 플랫폼의 이름입니다.

Year : 게임이 출시된 연도입니다.

Genre : 게임의 장르입니다.

Publisher : 게임을 배급한 회사입니다.

NA_Sales : 북미지역에서의 출고량입니다.

EU_Sales : 유럽지역에서의 출고량입니다.

JP_Sales : 일본지역에서의 출고량입니다.

Other_Sales : 기타지역에서의 출고량입니다

> # 1. EDA

1.중복데이터 확인

2.결측치 확인
  - 2.1 Year컬럼의 결측치 존재를 확인 그래서 게임 이름 같은 데이터터의 출시연도를 가져와 결측치를 대체함

3.필요없는 컬럼 Unnamed 삭제

4.숫자 데이터에 문자열 존재 여부 확인 후 제거 그리고 타입 변환
  - 4.1 각 지역 출고량에 K 와 M이 존재하여 K는 0.001로 대체하여 곱해주었고 M은 1로 대체하여 곱하였다.

5.각 지역의 모든 출고량을 합한 Sales_Sum 컬럼을 만듬


In [ ]:
df.info()
# 1. 중복 데이터 확인
print("\n중복 데이터 확인: ", df.duplicated().sum(),'\n')

# 2. 결측치 확인
print("결측치 확인: ",df.isnull().sum())

In [ ]:
#Unamed 삭제
df.drop('Unnamed: 0', axis=1, inplace=True)
df.info()

In [ ]:
#platform 데이터 확인
df.Platform.value_counts()

In [ ]:
#Genre 데이터 확인
df.Genre.value_counts()

In [ ]:
#Publisher 데이터 확인
df.Publisher.value_counts()

In [ ]:
df["NA_Sales"] = df.NA_Sales.replace({'[kK]': '*0.001', '[mM]': '*1'},  regex = True).map(pd.eval)
df["EU_Sales"] = df.EU_Sales.replace({'[kK]': '*0.001', '[mM]': '*1'},  regex = True).map(pd.eval)
df["JP_Sales"] = df.JP_Sales.replace({'[kK]': '*0.001', '[mM]': '*1'},  regex = True).map(pd.eval)
df["Other_Sales"] = df.Other_Sales.replace({'[kK]': '*0.001', '[mM]': '*1'},  regex = True).map(pd.eval)



df.NA_Sales = df.NA_Sales.astype('float')
df.EU_Sales = df.EU_Sales.astype('float')
df.JP_Sales = df.JP_Sales.astype('float')
df.Other_Sales = df.Other_Sales.astype('float')

df.info()

In [ ]:
na = np.array(df['NA_Sales'])
eu = np.array(df['EU_Sales'])
jp = np.array(df['JP_Sales'])
other = np.array(df['Other_Sales'])

df['Sales_Sum'] = na + eu + jp + other
df

In [ ]:
#결측치 삭제 데이터프레임
clear_df = df.dropna(axis=0)
clear_df.reset_index(drop=True)

#Year열의 결측치 중 게임 이름을 year_null에 저장
year_null = df.Name[df.Year.isnull()==True]
year_null = list(year_null)

#Year열의 결측치의 인덱스 값 year_null에 저장
year_null_index = df[df.Year.isnull()==True].index

#같은 게임이름이 있는 행의 날짜를 가져와 결측치를 채움
for i in range(len(year_null)):
  name_list = list(clear_df.Year[clear_df.Name == year_null[i]])
  if len(name_list) != 0:
    df.loc[year_null_index[i], 'Year'] = name_list[0]

#같은 이름이 없는 경우 행을 삭제 함. 
print("결측치의 비율이 매우 적으므로 행을 삭제", df.Year.isnull().sum() / len(df.Year))
df = df.dropna(axis=0)
df.reset_index(drop=True)
print("Year 결측치 개수: ",df.Year.isnull().sum())

#year데이터 int타입으로 변환환
df.Year=df.Year.astype('int')
print(df.info())


In [ ]:
#genre 결측치 같은 게임이름에서 장르를 추출하여 대체

genre_null = df.Name[df.Genre.isnull()==True]
print(genre_null)
genre_null = list(genre_null)
genre_null_index = df.Name[df.Genre.isnull()==True].index

for i in range(len(genre_null)):
  genre_list = list(clear_df.Genre[clear_df.Name == genre_null[i]])
  if len(genre_list) != 0:
    df.loc[genre_null_index[i], 'Genre'] = genre_list[0]

df.Genre.isnull().sum()

In [ ]:
df

In [ ]:
#publisher 결측치 같은 게임이름에서 publisher를 추출하여 대체

publisher_null = df.Name[df.Publisher.isnull()==True]
publisher_null = list(publisher_null)
publisher_null_index = df.Name[df.Publisher.isnull()==True].index

for i in range(len(publisher_null)):
  publisher_list = list(clear_df.Publisher[clear_df.Name == publisher_null[i]])
  if len(publisher_list) != 0:
    df.loc[publisher_null_index[i], 'Publisher'] = publisher_list[0]

df.Publisher.isnull().sum()

In [ ]:
df =  df.query("Year >= 1900")
df.Year.value_counts()

In [ ]:
df.groupby(['Genre']).sum()

> # 데이터분석


> ## Mission1 

1. 지역은 각 지역의 출고량으로 북미, 유럽, 일본, 기타 지역으로 구분하였다.

2. 각 지역별 선호하는 게임 장르는 북미는 Action, 유럽은 Action, 일본은 Role-Playing, 기타 다른 지역은 Action이며 각 지역별로 선호하는 게임장르가 일본을 제외한 유럽,북미,기타다른지역은 Ation으로 선호하는 장르가 같다.

3. 북미,유럽,기타지역등 action장르의 게임 출고량 랭킹을 봤을 때 북미,유럽,기타지역은 서로 연관되어 높게 나타나지만 일본은 출고량이 낮게 나타나고 
일본 Action게임의 최대 출고량을 봤을때도 나머지 지역들은 비교적 낮게 나온다. 

In [ ]:
#지역은 각 지역의 출고량으로 구별(NA, EU, JP, Other)

#지역 별로 선호하는 게임의 장르 
df_Genre = df.groupby('Genre')['NA_Sales','EU_Sales',	'JP_Sales', 'Other_Sales'].sum()
sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(df_Genre,annot=True,fmt='.3f')
df_Genre = df_Genre.reset_index()
df_Genre.plot.bar(x='Genre',figsize = (20, 10), fontsize = 20)


In [ ]:
#지역별 최고 수출량인 데이터 추출

df_Genre_sum = df.groupby('Genre')['NA_Sales','EU_Sales',	'JP_Sales', 'Other_Sales'].sum().reset_index()


NA_max = df_Genre_sum[df_Genre_sum.NA_Sales == max(df_Genre_sum.NA_Sales)]
EU_max = df_Genre_sum[df_Genre_sum.EU_Sales == max(df_Genre_sum.EU_Sales)]
JP_max = df_Genre_sum[df_Genre_sum.JP_Sales == max(df_Genre_sum.JP_Sales)]
Other_max = df_Genre_sum[df_Genre_sum.Other_Sales == max(df_Genre_sum.Other_Sales)]


display(NA_max[['Genre','NA_Sales']],EU_max[['Genre','EU_Sales']],
        JP_max[['Genre','JP_Sales']],Other_max[['Genre','Other_Sales']])

#각 지역별 선호하는 게임 장르는 북미는 Action, 유럽은 Action, 일본은 Role-Playing, 기타 다른 지역은 Action이며
#각 지역별로 선호하는 게임장르가 유럽,북미,기타다른지역은 Ation으로 선호하는 장르가 같다.



In [ ]:
df_action = df[df.Genre == 'Action']

In [ ]:
df_NA = pd.DataFrame(df_action.groupby(['Name','Genre'])['NA_Sales','EU_Sales','JP_Sales','Other_Sales'].sum())
df_NA_rank = df_NA.sort_values(ascending = False, by = 'NA_Sales').head(10)
df_EU_rank = df_NA.sort_values(ascending = False, by = 'EU_Sales').head(10)
df_JP_rank = df_NA.sort_values(ascending = False, by = 'JP_Sales').head(10)
df_Other_rank = df_NA.sort_values(ascending = False, by = 'Other_Sales').head(10)
df_NA_rank=df_NA_rank.reset_index()
display(df_NA_rank, df_EU_rank, df_JP_rank ,df_Other_rank)


#Action장르의 각 지역의 게임임 출고량 랭킹을 출력하였다. 북미,유럽,기타지역등 action장르의 게임임 출고량 랭킹을 봤을 때 2개의 지역 및 기타지역은 서로 연관되어 높게 나타나지만 일본은 출고량이 낮게 나타나고 
#일본 Action게임의 최대 출고량을 봤을때도 나머지 지역들은 비교적 낮게 나온다. 

> ## Mission 2
1. 연도별 게임의 출고량으로 트렌드 기준을 파악하였다.
2. 2000년대부터 액션 장르의 게임이 지속적으로 출고량이 높아 액션장르의 게임이 트렌드였다고 판단할 수 있다.
3. 2000년대부터 액션 장르의 출고량이 가장 높게 연속적으로 나타나므로 트렌드가 존재했다고 생각했다.

In [ ]:
df_year_genre = pd.DataFrame(columns = ['Year','Genre','NA_Sales_Sum','EU_Sales_Sum','JP_Sales_Sum','Other_Sales_Sum'])

# #연도별 장르별 
for i in range(len(df.Year.unique())):
  df_year_tmp=df[df.Year == df.Year.unique()[i]]
  for j in range(len(df_year_tmp.Genre.unique())):
    df_genre_tmp = df_year_tmp[df_year_tmp.Genre == df_year_tmp.Genre.unique()[j]]
    NA_sum = df_genre_tmp.NA_Sales.sum()
    EU_sum = df_genre_tmp.EU_Sales.sum()
    JP_sum = df_genre_tmp.JP_Sales.sum()
    Other_sum = df_genre_tmp.Other_Sales.sum()
    df_year_genre.loc[len(df_year_genre)] = [df.Year.unique()[i], df_year_tmp.Genre.unique()[j], NA_sum, EU_sum, JP_sum, Other_sum]

df_year_genre = df_year_genre.sort_values(ascending=True, by='Year')
df_year_genre['Sales_Sum'] = df_year_genre.NA_Sales_Sum + df_year_genre.EU_Sales_Sum + df_year_genre.JP_Sales_Sum +df_year_genre.Other_Sales_Sum
display(df_year_genre)

In [ ]:
a = pd.DataFrame(df_year_genre.groupby('Year')['Sales_Sum'].max()).reset_index()
df_year_max = pd.DataFrame(columns = ['Year', 'Genre', 'NA_Sales_Sum', 'EU_Sales_Sum', 'JP_Sales_Sum','Other_Sales_Sum', 'Sales_Sum'])
df_r1 = df_year_genre[df_year_genre.Year == a.Year.loc[1]]
df_r2 = df_r1[df_r1.Sales_Sum == a.Sales_Sum.loc[1]]


for i in range(len(a)):
  df_r1 = df_year_genre[df_year_genre.Year == a.Year.loc[i]]
  df_r2 = df_r1[df_r1.Sales_Sum == a.Sales_Sum.loc[i]]
  df_year_max = df_year_max.append(df_r2, ignore_index = True)
print(df_year_max.Genre.value_counts())
sns.set(rc = {'figure.figsize':(20,10)})
ax=sns.barplot(x='Year', y = 'Sales_Sum', data = df_year_max)
for i in range(len(df_year_max)):
  ax.text(x = i, y = 0 , s = str(df_year_max.Genre.loc[i]), ha = 'center', rotation = 90)

#2000년대부터 액션 장르의 게임이 지속적으로 출고량이 높아 액션장르의 게임이 트렌드였다고 판단할 수 있다.

> ## Mission3
1. 인기가 많다는 것은 각 지역의 최고 수출량을 기록한 게임이라고 정했습니다.
2. 최고 수출량을 기록한 Wii Sports게임이 인기가 있는 이유는 wii Sports게임의 플랫폼은 will이다. wii라는 플랫폼이 2006년 등장하여 트랜드를 가져갔으며 wii Sports는 가장 수출량이 높은 회사 'Nintendo'의 게임이기 때문에 인기가 많아졌다고 생각한다.
3. 총 게임별 판매량 순위 , 플랫폼별 총 판매량 순위, 회사별 총 판매량 순위,연도별 상위8개 플랫폼의 총 수출량 변화를 시각화하여 인기가 많은 게임의 이유를 찾아보려고 하였다.
4. 해당 시각화 자료를 통해 인기가 많은 게임의 이유를 알게 되었다.

In [ ]:
def Sales(columns_name1, columns_name2):
  df_NA = pd.DataFrame(df.groupby(columns_name1)[columns_name2].sum())
  return df_NA.sort_values(ascending = False, by = columns_name2 )

In [ ]:
df_NA = pd.DataFrame(df.groupby(['Name','Publisher'])['NA_Sales'].sum())
df_NA.sort_values(ascending = False, by = 'NA_Sales' )

In [ ]:
# 북미 최대 판매 게임 이름
df_NA_Name = Sales('Name', 'NA_Sales')
display(df_NA_Name.describe())
df_NA_Name_rank = df_NA_Name[df_NA_Name.NA_Sales > 0.3]
df_NA_Name_rank 

In [ ]:
# 북미 최대 판매 장르
df_NA_Genre = Sales('Genre', 'NA_Sales')
display(df_NA_Genre)

# 북미 최대 판매 게임 이름름
df_NA_Name = Sales('Name', 'NA_Sales')
display(df_NA_Name)

#북미 최대 판매 게임 회사
df_NA_Publisher = Sales('Publisher', 'NA_Sales')
display(df_NA_Publisher)

#북미 최대 판매 플랫폼
df_NA_Platform = Sales('Platform', 'NA_Sales')
display(df_NA_Platform)


In [ ]:
# 유럽 최대 판매 장르
df_EU_Genre = Sales('Genre', 'EU_Sales')
display(df_EU_Genre)

# 유럽 최대 판매 게임 이름름
df_EU_Name = Sales('Name', 'EU_Sales')
display(df_EU_Name)

# 유럽 최대 판매 게임 회사
df_EU_Publisher = Sales('Publisher', 'EU_Sales')
display(df_EU_Publisher)

# 유럽 최대 판매 플랫폼
df_EU_Platform = Sales('Platform', 'EU_Sales')
display(df_EU_Platform)

In [ ]:
# 일본 최대 판매 장르
df_JP_Genre = Sales('Genre', 'JP_Sales')
display(df_JP_Genre)

# 일본 최대 판매 게임 이름름
df_JP_Name = Sales('Name', 'JP_Sales')
display(df_JP_Name)

# 일본 최대 판매 게임 회사
df_JP_Publisher = Sales('Publisher', 'JP_Sales')
display(df_JP_Publisher)

# 일본 최대 판매 플랫폼
df_JP_Platform = Sales('Platform', 'JP_Sales')
display(df_JP_Platform)

In [ ]:
# 기타지역 최대 판매 장르
df_Other_Genre = Sales('Genre', 'Other_Sales')
display(df_Other_Genre)

# 기타지역 최대 판매 게임 이름름
df_Other_Name = Sales('Name', 'Other_Sales')
display(df_Other_Name)

# 기타지역 최대 판매 게임 회사
df_Other_Publisher = Sales('Publisher', 'Other_Sales')
display(df_Other_Publisher)

# 기타지역 최대 판매 플랫폼
df_Other_Platform = Sales('Platform', 'Other_Sales')
display(df_Other_Platform)

In [ ]:
# 총 게임별 판매량 순위
df_sales = pd.DataFrame(df.groupby('Name')['Sales_Sum'].sum())
df_sales = df_sales.sort_values(ascending=False, by= 'Sales_Sum').reset_index().head(8)

sns.barplot(y = 'Sales_Sum', x = 'Name', data = df_sales)

In [ ]:
df_platform = pd.DataFrame(df.groupby('Platform')['Sales_Sum'].sum())
df_platform_rank = df_platform.sort_values(ascending=False, by='Sales_Sum').reset_index()
df_platform_rank.head(10)
sns.barplot(y = 'Sales_Sum', x = 'Platform', data = df_platform_rank.head(10))

In [ ]:
df_pulblisher = pd.DataFrame(df.groupby('Publisher')['Sales_Sum'].sum())
df_pulblisher_rank = df_pulblisher.sort_values(ascending=False, by='Sales_Sum').reset_index()
df_pulblisher_rank.head(5)
sns.barplot(y = 'Sales_Sum', x = 'Publisher', data = df_pulblisher_rank.head(5))

In [ ]:
#연도별 판매량 순위
#wii platform이 나오게 된 연도가 2006년부터이며 그 해 wii sports라는 게임을 출시하게 된다.

df_wii = df[df.Name == 'Wii Sports']
df_plt = df[df.Platform == 'Wii']
df_2006 = df[df.Year == 2006]
df_2006 = df_2006.sort_values(ascending=False, by = 'Sales_Sum').reset_index()
df_2006_sports = df_2006[df_2006.Genre == "Sports"]
display(df_2006_sports)
# #wii Sports가 나온 연도
# display(df_wii)
# #wii 플랫폼이 처음 나온 연도 2006
# df_plt.sort_values(by='Year').head(1)

In [ ]:
df_p_r=df_platform_rank.head(8)
df_p_r

In [ ]:
# 연도별 상위8개 플랫폼의 총 수출량 변화
df_platform = pd.DataFrame(df.groupby(['Platform','Year'])['Sales_Sum'].sum())
df_platform_year = df_platform.sort_values(by='Year').reset_index()
df_platform_year = df_platform_year.query("(Platform == 'PS2') | (Platform == 'X360') | (Platform == 'PS3') | (Platform == 'Wii') | (Platform == 'DS') | (Platform == 'PS') | (Platform == 'GBA') | (Platform == 'PSP')")
df_platform_year.Platform.value_counts()
plt.xlim(1980, 2022)

sns.lineplot(y = 'Sales_Sum', x = 'Year', hue = 'Platform', data = df_platform_year)


In [ ]:
_

In [ ]:
df[df.Name == 'Wii Sports']

In [ ]:
#총 연도별 판매랑 순위
df_rank = df.sort_values(ascending=False, by='Sales_Sum')
display(df_rank.head(5))

# 총 게임별 판매량 순위
df_sales = pd.DataFrame(df.groupby('Name')['Sales_Sum'].sum())
display(df_sales.sort_values(ascending=False, by= 'Sales_Sum').head(5))

#연도별 판매량 순위
df_year_rank = pd.DataFrame(df.groupby(['Year','Name'])["Sales_Sum"].sum())
display(df_year_rank)

#최고 판매량 게임 회사

df_pulblisher = pd.DataFrame(df.groupby('Publisher')['Sales_Sum'].sum())
df_pulblisher_rank = df_pulblisher.sort_values(ascending=False, by='Sales_Sum')
display(df_pulblisher_rank.head(5))

#연도별 총 게임회사 판매량량

df_publisher = pd.DataFrame(df.groupby(['Publisher','Year'])['Sales_Sum'].sum())
df_df_publisher_year = df_publisher.sort_values( by='Year')
display(df_df_publisher_year)

#최고 판매량 플랫폼폼

df_platform = pd.DataFrame(df.groupby('Platform')['Sales_Sum'].sum())
df_platform_rank = df_platform.sort_values(ascending=False, by='Sales_Sum')
display(df_platform_rank.head(5))

#연도별 총 판매량 플랫폼

df_platform = pd.DataFrame(df.groupby(['Platform','Year'])['Sales_Sum'].sum())
df_platform_year = df_platform.sort_values( by='Year')
display(df_platform_year)

----------------------------------------------------


>  ## Mission4 (결론)
1. 다음분기에는 최근에도 인기가 좋았던 액션 장르의 게임을 설계하는 것이 좋다고 생각한다.